## Double check na limpeza de dados para subir para o banco

In [1]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv(r'C:\Users\laais\CDS_Python_do_DS_ao_DEV\df_products_hm.csv')

In [2]:
data['Composition'].unique()

array(['Shell: Cotton 99%, Spandex 1%',
       'Pocket lining: Polyester 65%, Cotton 35%',
       'Pocket lining: Cotton 100%', 'Shell: Cotton 100%',
       'Cotton 99%, Spandex 1%', 'Cotton 98%, Spandex 2%',
       'Lining: Polyester 100%', 'Shell: Cotton 98%, Spandex 2%',
       'Pocket: Cotton 100%',
       'Shell: Cotton 90%, Elastomultiester 8%, Spandex 2%',
       'Pocket lining: Polyester 80%, Cotton 20%', nan,
       'Cotton 78%, Polyester 21%, Spandex 1%'], dtype=object)

In [3]:
data = pd.read_csv(r'C:\Users\laais\CDS_Python_do_DS_ao_DEV\df_products_hm.csv')

#product_id
#apagando as linhas vazias e colocando como inteiro
data = data.dropna(subset=['product_id'])
data['product_id'] = data['product_id'].astype(int)

#product_name
#deixar minusculo e separado por _ igual o product_category
data['product_name'] = data['product_name'].apply( lambda x: x.replace(' ','_').lower())

#product_price
#substituir o $ por nada e colocar o tipo como float
data['product_price'] = data['product_price'].apply( lambda x: x.replace('$ ','')).astype(float)

#scrapy_datetime
#tranformar para o tipo data
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format = '%Y-%m-%d %H:%M:%S')

#style_id 
#transformar para inteiro
data['style_id'] = data['style_id'].astype(int)

#color_id   
#transformar para inteiro
data['color_id'] = data['color_id'].astype(int)

#color_name    
#deixar minusculo e separado por _ quando não for nulo
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ','_').lower() if pd.notnull(x) else x)

#Fit 
#deixar minusculo e separado por _ quando não for nulo
data['Fit'] = data['Fit'].apply(lambda x: x.replace(' ','_').lower() if pd.notnull(x) else x)

#Composition
#primeiro vou tirar as informações de pocket lining, Lining e Shell
#coloca o str para vetoriar as colunas e conseguir aplicar linha a linha
data = data[~data['Composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['Composition'].str.contains('Lining:', na=False)]
data = data[~data['Composition'].str.contains('Shell:', na=False)]
data = data[~data['Composition'].str.contains('Pocket:', na=False)]

#removendo duplicatas
data= data.drop_duplicates(subset=['product_id', 'product_category', 'product_name', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'Fit'], keep='last')

#resetando o index
data = data.reset_index(drop=True)

#quebrando a Composition na virgula
df1 = data['Composition'].str.split(',', expand=True)

#ordem das colunas de Composition cotton| polyester | spandex
#cria um df vazio do mesmo tamanho de data com todas as colunas que eu quero já organizadas
df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['cotton','polyester','spandex'])

#adicionando os valores de cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'

#juntando os dfs
df_ref = pd.concat([df_ref, df_cotton], axis=1)
#excluindo a primeira coluna vazia de cotton e deixando a ultima
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
#se tiver NaN vai substituir por 'Cotton 0%'
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

#polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

#juntando os dfs
df_ref = pd.concat([df_ref, df_polyester], axis=1)
#excluindo a primeira coluna vazia de polyester e deixando a ultima
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
#se tiver NaN vai substituir por 'Polyester 0%'
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

#spandex
df_spandex = df1.loc[df1[1].str.contains('Spandex', na=True), 1]
df_spandex.name = 'spandex'

#combinando spandex das colunas 1 e 2
df_spandex = df_spandex.combine_first(df1[2])

#juntando os dfs
df_ref = pd.concat([df_ref, df_spandex], axis=1)
#excluindo a primeira coluna vazia de spandex e deixando a ultima
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
#se tiver NaN vai substituir por 'Spandex 0%'
df_ref['spandex'] = df_ref['spandex'].fillna('Spandex 0%')

#final join
data = pd.concat([data,df_ref],axis=1)

#format composition data
#quero extrair só os numeros das colunas de composição
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['spandex'] = data['spandex'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)

#apagando colunas que não vou usar
data = data.drop(columns=['Composition'], axis=1)

#apagando duplicatas 
data = data.drop_duplicates()

In [4]:
data.dtypes

product_id                   int32
product_category            object
product_name                object
product_price              float64
scrapy_datetime     datetime64[ns]
style_id                     int32
color_id                     int32
color_name                  object
Fit                         object
cotton                     float64
polyester                  float64
spandex                    float64
dtype: object

In [5]:
data.columns

Index(['product_id', 'product_category', 'product_name', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'Fit',
       'cotton', 'polyester', 'spandex'],
      dtype='object')

In [6]:
df_ref.isna().sum()

cotton       0
polyester    0
spandex      0
dtype: int64

In [7]:
df_polyester.unique()

array([nan, ' Polyester 21%'], dtype=object)

In [8]:
df1[0].unique()

array(['Cotton 99%', 'Cotton 98%', nan, 'Cotton 78%'], dtype=object)

In [9]:
data.sample(20)

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,cotton,polyester,spandex
79,690449056,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,56,black,skinny_fit,0.98,0.00,0.02
34,690449022,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,22,gray,skinny_fit,0.98,0.00,0.02
67,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,43,light_denim_gray/trashed,skinny_fit,0.98,0.00,0.02
41,690449051,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,51,black_washed_out,skinny_fit,0.98,0.00,0.02
91,971061004,men_jeans_slim,slim_tapered_cropped_jeans,29.99,2022-06-21 16:26:02,971061,4,dark_denim_gray,slim_fit,0.99,0.00,0.01
47,690449051,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,51,dark_blue/washed_out,skinny_fit,0.98,0.00,0.02
70,690449056,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,56,light_denim_blue,skinny_fit,0.98,0.00,0.02
92,938875007,men_jeans_slim,slim_tapered_jeans,39.99,2022-06-21 16:26:02,938875,7,pale_denim_blue,slim_fit,0.99,0.00,0.01
32,690449022,men_jeans_ripped,skinny_jeans,39.99,2022-06-21 16:26:02,690449,22,light_denim_blue/trashed,skinny_fit,0.98,0.00,0.02
103,1004199001,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,1,light_gray,skinny_fit,0.99,0.00,0.01


# Armazenando no banco de dados

In [10]:
#salvando os dados limpos em um csv
data.to_csv(r'C:\Users\laais\CDS_Python_do_DS_ao_DEV\products_hm_cleaned.csv', index=False)

In [11]:
data = pd.read_csv(r'C:\Users\laais\CDS_Python_do_DS_ao_DEV\products_hm_cleaned.csv', index_col=None)

In [12]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,cotton,polyester,spandex
0,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,black,skinny_fit,0.99,0.0,0.01
1,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,light_denim_blue,skinny_fit,0.99,0.0,0.01
2,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,denim_blue,skinny_fit,0.99,0.0,0.01
3,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,white,skinny_fit,0.99,0.0,0.01
4,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,light_gray,skinny_fit,0.99,0.0,0.01


In [13]:
import sqlite3

In [14]:
#query para criar a tabela que vamos armazenar os dados
query_showroom_schema = """
CREATE TABLE vitrine(
    product_id       INTEGER,
    product_category TEXT,
    product_name     TEXT,
    product_price    REAL,
    scrapy_datetime  TEXT,
    style_id         INTEGER,
    color_id         INTEGER,
    color_name       TEXT,
    fit              TEXT,
    cotton           REAl,
    polyester        REAL,
    spandex          REAL
    )
"""

In [15]:
#criando e conectando ao database
#se não tiver um banco de dados criado ele vai criar
conn = sqlite3.connect('hm_db.sqlite')

In [16]:
#planejando a execução da query no bd
cursor = conn.execute(query_showroom_schema)

#fazendo a execução
conn.commit()

In [18]:
from sqlalchemy import create_engine

In [19]:
#conectando com o database
conn = create_engine('sqlite:///hm_db.sqlite', echo=False)

In [30]:
query = """
select * from vitrine;
"""

pd.read_sql_query(query,conn)

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,cotton,polyester,spandex


In [31]:
#inserindo os dados no banco de dados
data.to_sql('vitrine', con=conn, if_exists='append', index=False)

In [32]:
query = """
select * from vitrine;
"""

df = pd.read_sql_query(query,conn)
df

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,cotton,polyester,spandex
0,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,black,skinny_fit,0.99,0.00,0.01
1,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,light_denim_blue,skinny_fit,0.99,0.00,0.01
2,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,denim_blue,skinny_fit,0.99,0.00,0.01
3,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,white,skinny_fit,0.99,0.00,0.01
4,1004199004,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-21 16:26:02,1004199,4,light_gray,skinny_fit,0.99,0.00,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...
121,1013317012,men_jeans_joggers,hybrid_regular_tapered_joggers,44.99,2022-06-21 16:26:02,1013317,12,light_denim_blue,regular_fit,0.78,0.21,0.01
122,1013317012,men_jeans_joggers,hybrid_regular_tapered_joggers,44.99,2022-06-21 16:26:02,1013317,12,denim_blue,regular_fit,0.78,0.21,0.01
123,1013317012,men_jeans_joggers,hybrid_regular_tapered_joggers,44.99,2022-06-21 16:26:02,1013317,12,dark_gray,regular_fit,0.78,0.21,0.01
124,1013317012,men_jeans_joggers,hybrid_regular_tapered_joggers,44.99,2022-06-21 16:26:02,1013317,12,dark_blue,regular_fit,0.78,0.21,0.01


In [37]:
query = """
ALTER TABLE vitrine RENAME TO vitrine_two
"""

conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query)
#fazendo a execução
conn.commit()

In [38]:
#Criando um index
query = """
CREATE INDEX idx_product_id ON vitrine_two(product_id)
"""

conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query)
#fazendo a execução
conn.commit()